In [1]:
import json
data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]

In [2]:
print(repr(data))
print(json.dumps(data))

[{'a': 'A', 'b': (2, 4), 'c': 3.0}]
[{"a": "A", "b": [2, 4], "c": 3.0}]


In [3]:
decoded = json.loads(json.dumps(data))
print(data[0]['b'])
print(decoded[0]['b'])

(2, 4)
[2, 4]


In [4]:
data = [{'a': 'A', 'c': 3.0,'b': (2, 4), }]
unsorted = json.dumps(data)
first_sort = json.dumps(data, sort_keys=True)
print(unsorted)
print(first_sort)
print(unsorted == first_sort)

[{"a": "A", "c": 3.0, "b": [2, 4]}]
[{"a": "A", "b": [2, 4], "c": 3.0}]
False


In [5]:
print(json.dumps(data, sort_keys=True, indent=2))

[
  {
    "a": "A",
    "b": [
      2,
      4
    ],
    "c": 3.0
  }
]


In [7]:
print(json.dumps(data, separators=('!','@')))

[{"a"@"A"!"c"@3.0!"b"@[2!4]}]


In [8]:
print('repr(data)             :', len(repr(data)))

plain_dump = json.dumps(data)
print('dumps(data)            :', len(plain_dump))

small_indent = json.dumps(data, indent=2)
print('dumps(data, indent=2)  :', len(small_indent))

with_separators = json.dumps(data, separators=(',', ':'))
print('dumps(data, separators):', len(with_separators))

repr(data)             : 35
dumps(data)            : 35
dumps(data, indent=2)  : 73
dumps(data, separators): 29


In [9]:
data = [{'a': 'A', 'b': (2, 4), 'c': 3.0, ('d',): 'D tuple'}]

print('First attempt')
try:
    print(json.dumps(data))
except TypeError as err:
    print('ERROR:', err)

print()
print('Second attempt')
print(json.dumps(data, skipkeys=True))

First attempt
ERROR: keys must be a string

Second attempt
[{"a": "A", "b": [2, 4], "c": 3.0}]


目前为止所有的例子都是用的 Python 的内置类型，json 原生就支持它们。不过有时我们也想编码一些自定义类，这里我们有两种方式来实现它。

In [10]:
class MyObj:
    def __init__(self, s):
        self.s = s
        
    def __repr__(self):
        return f'<MyObj({self.s})>'
    
obj = MyObj('hahhaa sth')
try:
    print(json.dumps(obj))
except TypeError as err:
    print(err)

Object of type 'MyObj' is not JSON serializable


In [11]:
# 方式一编码
def convert_to_builtin_type(obj):
    print('default:' ,obj)
    d = {
        '__class__': obj.__class__.__name__
    }
    d.update(obj.__dict__)
    return d

print(json.dumps(obj, default=convert_to_builtin_type))

default: <MyObj(hahhaa sth)>
{"__class__": "MyObj", "s": "hahhaa sth"}


In [20]:
# 方式一解码
def dict_to_object(d):
    if '__class__' in d:
        class_name = d.pop('__class__')
        class_ = globals()[class_name]
        print(class_)
        args = {key: value for key,value in d.items()}
        inst = class_(**args)
    else:
        inst = d
    return inst
        
encoded_object = '''
    [{"s": "instance value goes here",
      "__class__": "MyObj"}]
    '''

myobj_instance = json.loads(
    encoded_object,
    object_hook=dict_to_object,
)
print(myobj_instance)

<class '__main__.MyObj'>
[<MyObj(instance value goes here)>]


In [22]:
encoder = json.JSONEncoder()
data = [{'a': 'A', 'b': (2, 4), 'c': 3.0}]
for p in encoder.iterencode(data):
    print(p)

[
{
"a"
: 
"A"
, 
"b"
: 
[2
, 4
]
, 
"c"
: 
3.0
}
]


In [24]:
# 方式二编码
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        d = {
            '__class__': obj.__class__.__name__
        }
        d.update(obj.__dict__)
        return d
print(MyEncoder().encode(obj))

{"__class__": "MyObj", "s": "hahhaa sth"}


In [25]:
# 方式二解码
class MyDecoder(json.JSONDecoder):
    def __init__(self):
        super().__init__(object_hook=self.dict_to_object)
        
    def dict_to_object(self, d):
        if '__class__' in d:
            class_name = d.pop('__class__')
            class_ = globals()[class_name]
            print(class_)
            args = {key: value for key,value in d.items()}
            inst = class_(**args)
        else:
            inst = d
        return inst
    
myobj_instance = MyDecoder().decode(encoded_object)
print(myobj_instance)

<class '__main__.MyObj'>
[<MyObj(instance value goes here)>]


之前所有的例子都假设所编码后的整个数据都可以一次性写入内存。对于巨大的数据来说，写入到类文件对象中更加好一些。 load() 和 dump() 函数则可以将数据写入到一个类文件对象中以读取或写入

In [26]:
import io
f = io.StringIO()
json.dump(data, f)
print(f.getvalue())

[{"a": "A", "b": [2, 4], "c": 3.0}]


In [29]:
print(type(f))

<class '_io.StringIO'>


In [30]:
json.load(f)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [31]:
f = io.StringIO('[{"a": "A", "c": 3.0, "b": [2, 4]}]')
print(json.load(f))

[{'a': 'A', 'c': 3.0, 'b': [2, 4]}]


json.tool 模块实现了一个命令行程序来格式化 JSON 数据使其在命令行下更易阅读

```
$ python3 -m json.tool --sort-keys example.json

[
    {
        "a": "A",
        "b": [
            2,
            4
        ],
        "c": 3.0
    }
]
```